# 2D Point Mass Walking Model with Two Force Actuators

This notebook demonstrates trajectory optimization for a simple walking model using InfiniteOpt.jl. The model has a point mass moving through space with two legs that can apply forces to propel it forward in a walking gait.

## Setup and Package Loading

In [ ]:
# Activate environment and load packages
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using Revise
using InfiniteOpt, Ipopt, Plots, Distributions, LinearAlgebra

## Step 1: Define Model Parameters and Variables

Here we set up the basic optimization model with InfiniteOpt and define:
- Physical parameters (gravity, damping, force penalties)
- The infinite time parameter using orthogonal collocation
- State variables (position px, py and velocities vx, vy)
- Control variables (forces F_trail and F_lead on each leg)
- Time scaling variable t_f
- Fixed contact points for trailing and leading legs
- Leg geometry expressions (vectors, lengths, unit vectors)
- Force components along each leg

In [ ]:
# Create the optimization model
model = InfiniteModel(Ipopt.Optimizer)

# Model parameters
c_fr = 0.05  # Force rate penalty coefficient (0.05 creates linear forces)
c_t = 5.0    # Time penalty coefficient
k_b = 0.0    # Damping coefficient
g = 1        # Gravity
ω0 = 0.3     # Initial angular velocity
α = 0.35     # Angle for step length calculation
y_0 = 0.95   # Initial leg length
sl = 2*y_0*sin(α)  # Step length

# Infinite time parameter with 101 collocation points
@infinite_parameter(model, t in [0, 1], num_supports=101, derivative_method = OrthogonalCollocation(2))

# State variables with bounds and initial guesses
@variable(model, px, Infinite(t), start = (t) -> value(sl)*t)  # x position
@variable(model, py >= 0.1, Infinite(t), start = (t) -> y_0 + cos(π*t)*.1)  # y position (must be positive)
@variable(model, vx, Infinite(t), start = (t) -> 1.0)  # x velocity
@variable(model, vy, Infinite(t), start = (t) -> 0.0)  # y velocity

# Force magnitude along each leg (must be positive - legs can only push)
@variable(model, F_trail >= 0, Infinite(t), 
          start = (t) -> 1.0 * cos(2π*t) * (t < 0.5 ? 1.0 : 0.0))  # trailing leg force
@variable(model, F_lead >= 0, Infinite(t), 
          start = (t) -> 1.0 * -sin(2π*t) * (t <= 0.5 ? 0 : 1.0))  # leading leg force

# Time scaling variable
@variable(model, 0.001 <= t_f <= 10, start = 1)

# Fixed leg contact positions
@finite_parameter(model, P_trail_x==0.0)   # Trailing leg x-position
@finite_parameter(model, P_trail_y==0.0)   # Trailing leg y-position
@finite_parameter(model, P_lead_x==sl)     # Leading leg x-position
@finite_parameter(model, P_lead_y==0.0)    # Leading leg y-position

# Leg vectors (from contact points to COM)
@expression(model, trail_leg_x, px - P_trail_x)
@expression(model, trail_leg_y, py - P_trail_y)
@expression(model, lead_leg_x, px - P_lead_x)
@expression(model, lead_leg_y, py - P_lead_y)

# Leg lengths
@expression(model, trail_leg_length, sqrt(trail_leg_x^2 + trail_leg_y^2))
@expression(model, lead_leg_length, sqrt(lead_leg_x^2 + lead_leg_y^2))

# Unit vectors along each leg
@expression(model, trail_unit_x, trail_leg_x / (trail_leg_length))
@expression(model, trail_unit_y, trail_leg_y / (trail_leg_length))
@expression(model, lead_unit_x, lead_leg_x / (lead_leg_length))
@expression(model, lead_unit_y, lead_leg_y / (lead_leg_length))

# Force components for each leg
@expression(model, Ftrail_x, F_trail * trail_unit_x)
@expression(model, Ftrail_y, F_trail * trail_unit_y)
@expression(model, Flead_x,  F_lead * lead_unit_x)
@expression(model, Flead_y,  F_lead * lead_unit_y)

# Total force components
@expression(model, Ftot_x, Ftrail_x + Flead_x)
@expression(model, Ftot_y, Ftrail_y + Flead_y)

# Leg lengthening velocities
@expression(model, trail_leg_velocity, 
           (px*vx + py*vy) / (trail_leg_length))
@expression(model, lead_leg_velocity, 
            (-P_lead_x*vx + px*vx -P_lead_y*vy + py*vy) / (lead_leg_length))

## Step 2: Compute Mechanical Power

Mechanical power is calculated as force magnitude times leg-lengthening velocity. Positive power occurs when the leg extends (doing positive work), while negative power occurs during leg compression (absorbing energy). We use slack variables to separate positive and negative power contributions.

In [ ]:
# Power slack variables for each leg (separated into positive and negative)
@variable(model, pospower_trail >= 0, Infinite(t))
@variable(model, negpower_trail <= 0, Infinite(t))
@variable(model, pospower_lead >= 0, Infinite(t))
@variable(model, negpower_lead <= 0, Infinite(t))

# Mechanical power expressions
@expression(model, mechpower_trail, F_trail * trail_leg_velocity)
@expression(model, mechpower_lead, F_lead * lead_leg_velocity)

## Step 3: Power Splitting Constraints

These constraints ensure that the positive and negative power slack variables correctly bound the actual mechanical power. This allows us to penalize positive work (energy generation) differently from negative work (energy absorption) in the objective function.

In [ ]:
# Power splitting constraints for trailing leg
@constraint(model, pospower_trail >= mechpower_trail)
@constraint(model, pospower_trail >= 0)
@constraint(model, negpower_trail <= mechpower_trail)
@constraint(model, negpower_trail <= 0)

# Power splitting constraints for leading leg
@constraint(model, pospower_lead >= mechpower_lead)
@constraint(model, pospower_lead >= 0)
@constraint(model, negpower_lead <= mechpower_lead)
@constraint(model, negpower_lead <= 0)

## Step 4: System Dynamics

These are the equations of motion for the point mass. Position derivatives equal velocity, and velocity derivatives equal force divided by mass (mass = 1 here) minus gravity in the y-direction. The t_f factor accounts for time scaling.

In [ ]:
# Kinematic and dynamic constraints
@constraint(model, ∂(px, t) == t_f * vx)
@constraint(model, ∂(py, t) == t_f * vy)
@constraint(model, ∂(vx, t) == t_f * (Ftot_x))
@constraint(model, ∂(vy, t) == t_f * (Ftot_y - g))

## Step 5: Force Rate Dynamics and Leg Length Constraints

To encourage smooth force profiles, we introduce force rate (Fdot) and force acceleration (Fddot) variables. The force acceleration is split into positive and negative components to allow for asymmetric penalties. We also enforce that forces can only be applied when the leg length is at or below its rest length (complementarity constraint).

In [ ]:
# Force rate variables
@variable(model, Fdot_trail, Infinite(t))
@variable(model, Fdot_lead, Infinite(t))

# Force acceleration variables (split into positive and negative components)
@variable(model, Fddot_trail_p >= 0, Infinite(t))
@variable(model, Fddot_trail_m >= 0, Infinite(t))
@variable(model, Fddot_lead_p >= 0, Infinite(t))
@variable(model, Fddot_lead_m >= 0, Infinite(t))

# Scaling parameters (currently set to 1)
@variable(model, fdot_scale == 1)
@variable(model, fddot_scale == 1)

# Force rate dynamics
@constraint(model, ∂(F_trail, t) == t_f * (Fdot_trail)/fdot_scale)
@constraint(model, ∂(F_lead, t) == t_f * (Fdot_lead)/fdot_scale)
@constraint(model, ∂(Fdot_trail, t) == t_f * ((Fddot_trail_p - Fddot_trail_m)/fddot_scale))
@constraint(model, ∂(Fdot_lead, t) == t_f * ((Fddot_lead_p - Fddot_lead_m)/fddot_scale))

# Leg length constraints: forces only active when leg length <= 1 (complementarity)
@constraint(model, F_trail * (trail_leg_length - 1) <= 0)
@constraint(model, F_lead * (lead_leg_length - 1) <= 0)

## Step 6: Boundary Conditions

We enforce periodic boundary conditions for a symmetric walking gait. The walker starts at the origin with the trailing leg on the ground, and ends one step length away with the same height and horizontal velocity. Forces transition smoothly: trailing leg force ends at zero, leading leg force starts at zero, and force rates are matched at the boundaries.

In [ ]:
# Initial conditions
@constraint(model, px(0) == 0)
@constraint(model, py(0) == y_0)
@constraint(model, vy(0) == 0)  # Zero vertical velocity at start

# Final conditions (symmetric gait)
@constraint(model, px(1) == sl)      # One full step
@constraint(model, py(1) == y_0)     # Same height at end
@constraint(model, vx(1) == vx(0))   # Same horizontal velocity
@constraint(model, vy(1) == 0)       # Same vertical velocity

# Force boundary conditions
@constraint(model, F_trail(1) == 0)
@constraint(model, F_lead(0) == 0)
@constraint(model, Fdot_lead(0) == Fdot_trail(1))
@constraint(model, Fdot_trail(0) == Fdot_lead(1))

# Fix step time
@constraint(model,t_f == 1.2)

## Step 7: Define Objective Function

The objective minimizes a combination of:
1. Net mechanical work (positive work - negative work)
2. Force rate (to encourage smooth force profiles)
3. Time (optional, currently weighted)

This encourages efficient, smooth gaits that minimize energy expenditure.

In [ ]:
# Cost components
@expression(model, cost_work, 
    integral(pospower_trail, t) * t_f + integral(pospower_lead, t) * t_f - 
    integral(negpower_trail, t) * t_f - integral(negpower_lead, t) * t_f)

@expression(model, cost_fr, 
    c_fr*integral(Fddot_trail_p,t) * t_f + c_fr*integral(Fddot_lead_p,t)*t_f + 
    c_fr*integral(Fddot_trail_m,t) * t_f + c_fr*integral(Fddot_lead_m,t)*t_f)

@expression(model, cost_fr2, 
    c_fr*integral(Fddot_trail_p.^2,t) * t_f + c_fr*integral(Fddot_lead_p.^2,t)*t_f + 
    c_fr*integral(Fddot_trail_m.^2,t) * t_f + c_fr*integral(Fddot_lead_m.^2,t)*t_f)

@expression(model, cost_time, c_t*t_f)

# Objective: minimize work + force rate
@objective(model, Min, cost_work + cost_fr)

## Step 8: Configure Solver and Optimize

We use Ipopt as the nonlinear programming solver with warm starting enabled and moderate tolerance settings. The optimization typically takes 1-2 minutes depending on hardware.

In [ ]:
# Set solver parameters
set_optimizer_attribute(model, "max_cpu_time", 120.0)
set_optimizer_attributes(model, "tol" => 1e-3, "max_iter" => 500)
set_optimizer_attribute(model, "warm_start_init_point", "yes")

# Solve the optimization problem
optimize!(model)

## Step 9: Visualize Results

We create a comprehensive visualization showing:
1. Trajectory in x-y space
2. Position vs time
3. Velocities
4. Forces
5. Leg lengths
6. Force rates
7. Leg velocities
8. Dynamics constraint violations

In [ ]:
# Plotting function
function plot_results()
    f = plot(layout = (4,2), size = (800, 800))
    t_ = value(t)*value(t_f)
    txt="work:"*string(round(value(cost_work),digits=2)) * " force rate:" * string(round(value(cost_fr),digits=2))
    plot!(value(px),value(py),subplot = 1,xlabel="x",ylabel="y",title=txt)
    plot!(t_,value(px),subplot = 2,ylabel="pxy",xlabel="time")
    plot!(t_,value(py),subplot = 2)
    plot!(t_,value(vx),subplot=3,label="vx",ylabel="vel")
    plot!(t_,value(vy),subplot=3,label="vy")
    plot!(t_,value(Ftrail_x), label="Ftrail_x",subplot=4,ylabel="force")
    plot!(t_,value(Ftrail_y), label="Ftrail_y",subplot=4)
    plot!(t_,value(Flead_y) + value(Ftrail_y), label="FY",subplot=4)
    plot!(t_,value(Flead_x), label="Flead_x",subplot=4)
    plot!(t_,value(Flead_y), label="Flead_y",subplot=4)

    plot!(t_,value(trail_leg_length),subplot=5,label="trail",linewidth=2,ylabel="leg length")
    plot!(t_,sqrt.((value(px) .- value(P_trail_x)).^2 + (value(py) .- value(P_trail_y)).^2),subplot=5,label="trailcomp")
    plot!(t_,value(lead_leg_length),subplot=5,label="lead")

    # plot fdotdot for each leg
    plot!(t_,value(Fddot_trail_p)-value(Fddot_trail_m),subplot=6,label="trail",ylabel="force rate")
    plot!(t_,value(Fddot_lead_p)-value(Fddot_lead_m),subplot=6,label="lead")

    plot!(t_,value(trail_leg_velocity),subplot=7,label = "trail",ylabel="dlegdt")
    plot!(t_,value(lead_leg_velocity),subplot=7,label = "lead",ylabel="dlegdt")

    vx_dot = value(∂(vx,t))
    vy_dot = value(∂(vy,t))
    force_viol_x = vx_dot - value(Ftot_x)*value(t_f)
    force_viol_y = vy_dot - (value(Ftot_y) .- g)*value(t_f)
    plot!(t_,force_viol_x,subplot=8,label="dyn_viol_x")
    plot!(t_,force_viol_y,subplot=8,label="dyn_viol_y",xlabel="time")
    return f
end

# Generate and display the plot
f = plot_results()
display(f)

## Step 10: Animate the Walking Motion

This creates a simple animation showing the walker's center of mass and leg configurations over time. The animation shows the point mass (red circle) connected to ground contact points by lines representing the legs.

In [ ]:
# Extract trajectory data
px_vals = value(px)
py_vals = value(py)
trail_x = value(P_trail_x)
trail_y = value(P_trail_y)
lead_x = value(P_lead_x)
lead_y = value(P_lead_y)

# Create animation
anim = @animate for i in 1:length(px_vals)
    plot(xlim=(-0.2, sl+0.2), ylim=(-0.1, 1.5), aspect_ratio=:equal, 
         legend=false, xlabel="x", ylabel="y", title="Walking Model Animation")
    
    # Draw ground
    plot!([-0.2, sl+0.2], [0, 0], color=:black, linewidth=2)
    
    # Draw legs
    plot!([trail_x, px_vals[i]], [trail_y, py_vals[i]], 
          color=:blue, linewidth=3, label="trailing leg")
    plot!([lead_x, px_vals[i]], [lead_y, py_vals[i]], 
          color=:green, linewidth=3, label="leading leg")
    
    # Draw contact points
    scatter!([trail_x, lead_x], [trail_y, lead_y], 
             color=:black, markersize=6, markershape=:square)
    
    # Draw center of mass
    scatter!([px_vals[i]], [py_vals[i]], 
             color=:red, markersize=10)
    
    # Draw trajectory trace
    plot!(px_vals[1:i], py_vals[1:i], 
          color=:red, alpha=0.3, linewidth=1)
end

# Display the animation (adjust fps as needed)
gif(anim, "walking_animation.gif", fps=15)